# Google Colab Setup

**Note**: If running on Google Colab, run the cell below to download data first.

**註**: 如在 Google Colab 執行，請先執行下方 cell 下載資料。

In [ ]:
# Google Colab: Download dataset
# 下載資料集（僅 Colab 需要）
import os

if 'COLAB_GPU' in os.environ or not os.path.exists('../data'):
    print('Downloading dataset from GitHub...')
    print('下載資料集...')
    
    # Clone repository
    !git clone --depth 1 https://github.com/thc1006/NYCU-Campus-3D-Buildings.git
    
    # Change to repository directory
    os.chdir('NYCU-Campus-3D-Buildings/examples')
    
    print('Done! / 完成！')
else:
    print('Running locally / 本地執行')

# NQSD 範例 3：互動式視覺化本範例示範各種視覺化技術：- 2D 互動式地圖（Folium）- 建築高度熱力圖- 3D 視覺化（Plotly）- 匯出為 HTML**資料來源**：- NLSC 3D Building Data- OpenStreetMap (© OSM contributors)

In [ ]:
# 匯入必要套件import geopandas as gpdimport foliumfrom folium import pluginsimport matplotlib.pyplot as pltimport matplotlib.cm as cmimport matplotlib.colors as colorsprint("OK 套件載入成功")

## 1. 載入資料

In [ ]:
# 讀取合併資料集buildings = gpd.read_file('../data/output/latest/buildings_merged.geojson')# 轉換為 WGS84（地圖顯示用）if buildings.crs != 'EPSG:4326':    buildings = buildings.to_crs('EPSG:4326')# 計算校區中心點（用於地圖初始位置）center_lat = buildings.geometry.centroid.y.mean()center_lon = buildings.geometry.centroid.x.mean()print(f"OK 已載入 {len(buildings)} 棟建築")print(f"校區中心：{center_lat:.4f}, {center_lon:.4f}")

## 2. 基礎互動式地圖

In [ ]:
# 建立基礎地圖m = folium.Map(    location=[center_lat, center_lon],    zoom_start=15,    tiles='OpenStreetMap')# 僅顯示有名稱的建築（避免太多資料）named_buildings = buildings[buildings['name'].notna()].copy()# 新增建築標記for idx, row in named_buildings.iterrows():    # 取得建築中心點    if row.geometry.geom_type == 'Polygon':        centroid = row.geometry.centroid    else:        centroid = row.geometry        # 建立 popup 內容    popup_html = f"""    <b>{row['name']}</b><br>    English: {row.get('name:en', 'N/A')}<br>    高度: {row['nlsc_BUILD_H']:.1f} m<br>    結構: {row.get('nlsc_BUILD_STR', 'N/A')}    """        # 新增標記    folium.Marker(        location=[centroid.y, centroid.x],        popup=folium.Popup(popup_html, max_width=200),        tooltip=row['name']    ).add_to(m)# 顯示地圖m

## 3. 建築高度分色地圖

In [ ]:
# 建立地圖m2 = folium.Map(    location=[center_lat, center_lon],    zoom_start=15,    tiles='CartoDB positron')# 建立顏色對應函數（依高度）min_height = buildings['nlsc_BUILD_H'].min()max_height = buildings['nlsc_BUILD_H'].max()colormap = cm.LinearColormap(    colors=['green', 'yellow', 'orange', 'red'],    vmin=min_height,    vmax=max_height,    caption='建築高度（公尺）')# 新增建築多邊形（僅顯示有輪廓的建築）polygon_buildings = buildings[buildings.geometry.geom_type == 'Polygon'].copy()for idx, row in polygon_buildings.iterrows():    # 根據高度決定顏色    color = colormap(row['nlsc_BUILD_H'])        # Popup 內容    popup_html = f"""    <b>{row.get('name', '未命名建築')}</b><br>    高度: {row['nlsc_BUILD_H']:.1f} m<br>    結構: {row.get('nlsc_BUILD_STR', 'N/A')}<br>    ID: {row.get('nlsc_BUILD_ID', 'N/A')}    """        # 新增多邊形    folium.GeoJson(        row.geometry,        style_function=lambda x, color=color: {            'fillColor': color,            'color': 'black',            'weight': 1,            'fillOpacity': 0.6        },        tooltip=row.get('name', f"高度: {row['nlsc_BUILD_H']:.1f}m"),        popup=folium.Popup(popup_html, max_width=250)    ).add_to(m2)# 新增顏色圖例colormap.add_to(m2)# 顯示地圖m2

## 4. 建築高度熱力圖

In [ ]:
# 建立熱力圖資料heat_data = []for idx, row in buildings.iterrows():    if row.geometry.geom_type == 'Polygon':        centroid = row.geometry.centroid    else:        centroid = row.geometry        # [緯度, 經度, 權重（高度）]    heat_data.append([centroid.y, centroid.x, row['nlsc_BUILD_H']])# 建立地圖m3 = folium.Map(    location=[center_lat, center_lon],    zoom_start=15,    tiles='CartoDB dark_matter')# 新增熱力圖圖層plugins.HeatMap(    heat_data,    min_opacity=0.2,    max_zoom=18,    radius=25,    blur=20,    gradient={        0.0: 'blue',        0.3: 'lime',        0.5: 'yellow',        0.7: 'orange',        1.0: 'red'    }).add_to(m3)# 顯示地圖m3

## 5. 圖層控制地圖

In [ ]:
# 建立地圖with 多個底圖選項m4 = folium.Map(    location=[center_lat, center_lon],    zoom_start=15)# 新增不同底圖folium.TileLayer('OpenStreetMap', name='街道圖').add_to(m4)folium.TileLayer('CartoDB positron', name='淺色圖').add_to(m4)folium.TileLayer('CartoDB dark_matter', name='深色圖').add_to(m4)# 建立不同圖層# 圖層 1：所有建築all_buildings_layer = folium.FeatureGroup(name='所有建築')for idx, row in polygon_buildings.iterrows():    folium.GeoJson(        row.geometry,        style_function=lambda x: {            'fillColor': 'blue',            'color': 'black',            'weight': 1,            'fillOpacity': 0.5        }    ).add_to(all_buildings_layer)all_buildings_layer.add_to(m4)# 圖層 2：高樓建築（>30m）tall_buildings = buildings[buildings['nlsc_BUILD_H'] > 30]tall_buildings_layer = folium.FeatureGroup(name='高樓建築 (>30m)')for idx, row in tall_buildings.iterrows():    if row.geometry.geom_type == 'Polygon':        centroid = row.geometry.centroid    else:        centroid = row.geometry        folium.CircleMarker(        location=[centroid.y, centroid.x],        radius=8,        color='red',        fill=True,        fillColor='red',        fillOpacity=0.7,        popup=f"{row.get('name', 'N/A')}: {row['nlsc_BUILD_H']:.1f}m"    ).add_to(tall_buildings_layer)tall_buildings_layer.add_to(m4)# 新增圖層控制folium.LayerControl().add_to(m4)# 顯示地圖m4

## 6. 匯出為 HTML

In [ ]:
# 匯出不同地圖os.makedirs('outputs', exist_ok=True)m.save('outputs/map_basic.html')m2.save('outputs/map_height_colored.html')m3.save('outputs/map_heatmap.html')m4.save('outputs/map_layered.html')print("OK 所有地圖已匯出至 outputs/ 目錄：")print("  - map_basic.html（基礎地圖）")print("  - map_height_colored.html（高度分色）")print("  - map_heatmap.html（熱力圖）")print("  - map_layered.html（圖層控制）")

## 7. 統計圖表

In [ ]:
# 建立綜合統計圖表fig, axes = plt.subplots(2, 2, figsize=(14, 10))plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']# 1. 高度分佈axes[0, 0].hist(buildings['nlsc_BUILD_H'], bins=30, edgecolor='black', color='skyblue')axes[0, 0].set_xlabel('建築高度（公尺）')axes[0, 0].set_ylabel('建築數量')axes[0, 0].set_title('建築高度分佈')axes[0, 0].grid(True, alpha=0.3)# 2. 結構類型structure_counts = buildings['nlsc_BUILD_STR'].value_counts()axes[0, 1].bar(range(len(structure_counts)), structure_counts.values, color='coral', edgecolor='black')axes[0, 1].set_xticks(range(len(structure_counts)))axes[0, 1].set_xticklabels(structure_counts.index)axes[0, 1].set_xlabel('結構類型')axes[0, 1].set_ylabel('建築數量')axes[0, 1].set_title('建築結構類型')axes[0, 1].grid(True, alpha=0.3, axis='y')# 3. 高度分級bins = [0, 10, 20, 30, 40, 50, 100]labels = ['0-10m', '10-20m', '20-30m', '30-40m', '40-50m', '50m+']height_cats = pd.cut(buildings['nlsc_BUILD_H'], bins=bins, labels=labels)cat_counts = height_cats.value_counts().sort_index()axes[1, 0].pie(cat_counts, labels=cat_counts.index, autopct='%1.1f%%', startangle=90)axes[1, 0].set_title('建築高度分級比例')# 4. Top 10 最高建築named = buildings[buildings['name'].notna()]top_10 = named.nlargest(10, 'nlsc_BUILD_H')axes[1, 1].barh(range(len(top_10)), top_10['nlsc_BUILD_H'], color='steelblue', edgecolor='black')axes[1, 1].set_yticks(range(len(top_10)))axes[1, 1].set_yticklabels(top_10['name'], fontsize=8)axes[1, 1].set_xlabel('建築高度（公尺）')axes[1, 1].set_title('Top 10 最高建築')axes[1, 1].invert_yaxis()axes[1, 1].grid(True, alpha=0.3, axis='x')plt.tight_layout()os.makedirs('outputs', exist_ok=True)plt.savefig('outputs/statistics_charts.png', dpi=300, bbox_inches='tight')print("OK 統計圖表已儲存為 outputs/statistics_charts.png")plt.show()

## 總結本範例完成了：- [OK] 基礎互動式地圖（Folium）- [OK] 建築高度分色地圖- [OK] 建築高度熱力圖- [OK] 多圖層控制地圖- [OK] 匯出為 HTML 檔案- [OK] 綜合統計圖表所有地圖已匯出至 `outputs/` 目錄，可在瀏覽器中開啟查看！**使用建議**：- 在瀏覽器中開啟 HTML 地圖- 可縮放、平移、點擊建築查看資訊- 切換不同圖層觀察不同面向